# Lesson 2 Guidelines for Prompting
[Course](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/2/guidelines)

## Principles of Prompting
1. Write clear and specific prompts.
2. Give the model time to "think".

## Setup
### Install dependencies

In [1]:
!pip install openai

     |████████████████████████████████| 71 kB 691 kB/s eta 0:00:011
     |████████████████████████████████| 338 kB 14.3 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 11.1 MB/s eta 0:00:01


### Load API key and relevant libraries

In [4]:
!pip install python-dotenv

In [7]:
import openai
import os

from dotenv import load_dotenv, find_dotenv

# load ENVs from a local .env file
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')
model = "gpt-3.5-turbo"

### helper function

We'll be using the ChatGPT model "gpt-3.5-turbo" and OpenAI's [Chat completion endpoint](https://platform.openai.com/docs/guides/chat).

In [31]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = 0, # the degree of randomness of the model's output
    )
    # print(response)
    return response.choices[0].message["content"]
    # return response

## Principle 1: Write clear and specific instructions

### Tactics
1. Use delimiters to clearly indicate distinct parts of the input
2. Ask for a structured output
3. Ask the model to check whether conditions are satisfied
4. "Few-shot" prompting

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ` ``` `, `"""`, `< >`, `<tag></tag>`, `:`

In [13]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks \
    into a single sentence.
```{text}```
"""

response = get_completion(prompt)
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "To guide a model towards the desired output and reduce the chances of irrelevant or incorrect responses, it is important to provide clear and specific instructions, which may require longer prompts for more clarity and context.",
        "role": "assistant"
      }
    }
  ],
  "created": 1683081517,
  "id": "chatcmpl-7Bwg1sXSNIfucROYXl3TY1AiOOTCx",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 39,
    "prompt_tokens": 134,
    "total_tokens": 173
  }
}
To guide a model towards the desired output and reduce the chances of irrelevant or incorrect responses, it is important to provide clear and specific instructions, which may require longer prompts for more clarity and context.


### Tactic 2: Ask for a structured output
- JSON, HTML

In [21]:
prompt = f"""
Generate a list of three Tolkien's books with the title, genre, first published date and city. \
    Provide them in markdown table format with the following table headers:
    book_id, title, author, genre, publish_date, publish_city.
"""

response = get_completion(prompt)
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "| book_id | title | author | genre | publish_date | publish_city |\n|---------|-------|--------|-------|--------------|--------------|\n| 1 | The Hobbit | J.R.R. Tolkien | Fantasy | September 21, 1937 | London |\n| 2 | The Lord of the Rings | J.R.R. Tolkien | Fantasy | July 29, 1954 | London |\n| 3 | The Silmarillion | J.R.R. Tolkien | Fantasy | September 15, 1977 | London |",
        "role": "assistant"
      }
    }
  ],
  "created": 1683082591,
  "id": "chatcmpl-7BwxLSlbynMtUlwXK4D4KDNcXSnsW",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 110,
    "prompt_tokens": 58,
    "total_tokens": 168
  }
}


In [23]:
print(response.choices[0].message["content"])

| book_id | title | author | genre | publish_date | publish_city |
|---------|-------|--------|-------|--------------|--------------|
| 1 | The Hobbit | J.R.R. Tolkien | Fantasy | September 21, 1937 | London |
| 2 | The Lord of the Rings | J.R.R. Tolkien | Fantasy | July 29, 1954 | London |
| 3 | The Silmarillion | J.R.R. Tolkien | Fantasy | September 15, 1977 | London |


| book_id | title | author | genre | publish_date | publish_city |
|---------|-------|--------|-------|--------------|--------------|
| 1 | The Hobbit | J.R.R. Tolkien | Fantasy | September 21, 1937 | London |
| 2 | The Lord of the Rings | J.R.R. Tolkien | Fantasy | July 29, 1954 | London |
| 3 | The Silmarillion | J.R.R. Tolkien | Fantasy | September 15, 1977 | London |

### Tactic 3: Ask the model to check whether conditions are satisfied
We let the model extract steps if the input text contains a sequence of instructions, otherwise output "no steps provided".

In [24]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some water boiling. \
    While that's happening, grab a cup and put a tea bag in it. \
    Once the water is hot enough, just pour it over the tea bag. \
    Let it sit for a bit so the tea can steep. 
    After a few minutes, take out the tea bag. \
    If you like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious cup of tea to enjoy.
"""

prompt = f"""
You will be provided with text delimited by triple quotes. \
    If it contains a sequence of instructions, \
    re-write those instructions in the following format:
    
Step 1 - ...
Step 2 - ...
...
Step N - ...

If the text does not contain a sequence of instructions, \
    then simply write \"No steps provided.\"
    
\"\"\"{text_1}\"\"\"
"""

response = get_completion(prompt)

In [26]:
print("Completion for Text 1:")
print(response.choices[0].message.content)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea!


In [27]:
text_2 = text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""

prompt = f"""
You will be provided with text delimited by triple quotes. \
    If it contains a sequence of instructions, \
    re-write those instructions in the following format:
    
Step 1 - ...
Step 2 - ...
...
Step N - ...

If the text does not contain a sequence of instructions, \
    then simply write \"No steps provided.\"
    
\"\"\"{text_2}\"\"\"
"""

response = get_completion(prompt)

In [28]:
print("Completion for Text 2:")
print(response.choices[0].message.content)

Completion for Text 2:
No steps provided.


### Tactic 4: "Few-shot" prompting
Give it example(s) to follow.

In [30]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that arves the deepest valley flows from a modest spring;
    the grandest symphony originates from a single note;
    the most intricate tapestry begins with a solitary thread.
    
<child>: Teach me about resilience.
"""

response = get_completion(prompt)
print(response.choices[0].message.content)

<grandparent>: Resilience is like a tree that bends with the wind but does not break. It is the ability to bounce back from adversity and overcome challenges. Just like a tree needs strong roots to withstand the storm, we need to cultivate inner strength and perseverance to face life's obstacles. Remember, every setback is an opportunity to grow stronger and wiser.


## Principle 2: Give the model time to "think"

If a model is making reasoning errors by rushing to an incorrect conclusion, you should try reframing the query 
to request a chain or series of relevant reasoning before the model provides its final answer.

### Tactics
1. Specify the steps required to complete a task
2. Instruct the model to work out its own solution before rushing to a conclusion


#### Tactic 1: Specify the steps required to complete a task

In [33]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""

# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple backticks with 1 sentence
2 - Translate the summary into Japanese
3 - List each name in the Japanese summary
4 - Output a json object that contains the following keys: japenese_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Two siblings, Jack and Jill, go on a quest to fetch water from a well on a hilltop, but misfortune strikes and they both tumble down the hill, returning home slightly battered but with their adventurous spirits undimmed.

2つの兄弟、ジャックとジルは、丘の上の井戸から水を取るためにクエストに出かけますが、不幸が襲い、両方が丘を転がり落ち、少し傷ついて家に帰りますが、彼らの冒険心は減退しませんでした。

ジャック、ジル

{
  "japanese_summary": "2つの兄弟、ジャックとジルは、丘の上の井戸から水を取るためにクエストに出かけますが、不幸が襲い、両方が丘を転がり落ち、少し傷ついて家に帰りますが、彼らの冒険心は減退しませんでした。",
  "num_names": 2
}


Now let's give it a better prompt with format instructions.

In [34]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""

# example 2
prompt_2 = f"""
Perform the following actions:
1 - Summarize the following text delimited by <> backticks with 1 sentence
2 - Translate the summary into Chinese
3 - List each name in the Chinese summary
4 - Output a json object that contains the following keys: chinese_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in the translated summary>
Output JSON: <json with summary and num_names>

Text:
<{text}>
"""

response = get_completion(prompt_2)
print("Completion for prompt 2:")
print(response)

Completion for prompt 2:
Summary: Siblings Jack and Jill go on a quest to fetch water from a hilltop well, but misfortune strikes and they tumble down the hill, returning home slightly battered but with undimmed adventurous spirits. 
Translation: 兄妹杰克和吉尔前往山顶井取水，但不幸摔倒滚下山，虽然受了点伤，但他们的冒险精神仍然不减，继续愉快地探索。
Names: Jack, Jill
Output JSON: {"chinese_summary": "兄妹杰克和吉尔前往山顶井取水，但不幸摔倒滚下山，虽然受了点伤，但他们的冒险精神仍然不减，继续愉快地探索。", "num_names": 2}


#### Tactic 2: Instruct the model to work out its own solotion before rushing to a conclusion 

In [35]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct.


The student's solution is actually incorrect. So let's ask the model to work out its own solution first, then compare with the student's solution to determine if the student's solution is correct.

In [41]:
prompt = f"""
You are a very strict teacher. Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Output the question, your solution, the student's solution, the comparison, and the student's grade.
Use the following format:

- Question: the question

- Correct solution: the correct solution

- Student's solution: the student's solution

- Comparison:

- Grade: 100 if the solution is correct, 0 if incorrect


Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Output:
"""
response = get_completion(prompt)
print(response)

- Question: What is the total cost for the first year of operations as a function of the number of square feet?

- Correct solution: Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000

- Student's solution: Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000

- Comparison: The student's solution is incorrect because they added $100 per square foot for maintenance instead of $10 per square foot.

- Grade: 0


### Model Limitations

#### Hallucinations
Models sometimes make statements that sound plausible but are not true.

##### To reduce hallucinations:
Tirst ask the model to find relevant information, then answer the question based on the relevant information.

In [39]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Gillette
"""

response = get_completion(prompt)
print(response)

As an AI language model, I don't have personal experience or emotions, but I can provide you with information about the AeroGlide UltraSlim Smart Toothbrush by Gillette. 

The AeroGlide UltraSlim Smart Toothbrush is a high-tech toothbrush that uses advanced technology to provide a superior brushing experience. It features a slim, lightweight design that makes it easy to hold and maneuver, and it uses sonic vibrations to clean teeth and remove plaque. 

The toothbrush also comes with a companion app that provides real-time feedback on your brushing technique, helping you to improve your oral hygiene habits. The app tracks your brushing time, pressure, and coverage, and it provides personalized recommendations for improving your brushing technique. 

The AeroGlide UltraSlim Smart Toothbrush also has a long battery life, lasting up to two weeks on a single charge. It comes with a charging base that doubles as a storage case, making it easy to take with you on the go. 

Overall, the AeroGl

In [40]:
prompt = f"""
Find relevant information on Gillette. Then tell me about AeroGlide UltraSlim Smart Toothbrush by Gillette citing your sources.
"""

response = get_completion(prompt)
print(response)

Gillette is a brand that is primarily known for its shaving products, including razors, shaving creams, and aftershaves. The brand was founded in 1901 by King C. Gillette and has since become a household name in the personal care industry.

In recent years, Gillette has expanded its product line to include other personal care items, such as deodorants and body washes. One of the brand's newest products is the AeroGlide UltraSlim Smart Toothbrush.

According to Gillette's website, the AeroGlide UltraSlim Smart Toothbrush is designed to provide a superior clean with its advanced sonic technology. The toothbrush features a slim, ergonomic handle and a brush head that is designed to reach every corner of the mouth.

The toothbrush also comes with a mobile app that allows users to track their brushing habits and receive personalized recommendations for improving their oral hygiene. The app also features a timer that helps users ensure they are brushing for the recommended two minutes.

Over